In [1]:
import train_test_generation
from keras.layers import Dense,Conv2D,MaxPooling2D,UpSampling2D
from keras import Input, Model
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import regularizers
import keras
import pandas as pd

62.5


In [3]:
features = pd.read_csv('iterative_dataset.csv')

In [5]:
features.head()

,Unnamed: 0,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi,cold_resistance,scoring_difference,thermal,hoodie,fleece,wool,light_down,thick_down
0,0,4,26,23,3,58,73.425545,162.931325,1,41.130917,27.659097,70.0,-17.2,0,0,0,0,0,0
1,1,4,26,23,3,58,73.425545,162.931325,1,41.130917,27.659097,70.0,-17.2,0,0,0,0,0,1
2,2,4,26,23,3,58,73.425545,162.931325,1,41.130917,27.659097,70.0,-17.2,0,0,0,0,1,0
3,3,4,26,23,3,58,73.425545,162.931325,1,41.130917,27.659097,70.0,-17.2,0,0,0,0,1,1
4,4,4,26,23,3,58,73.425545,162.931325,1,41.130917,27.659097,70.0,-17.2,0,0,0,1,0,0


In [6]:
user_params_df = features[['age', 'weight', 'height', 'sex', 'fatpercentage', 'bmi']]

In [12]:
user_params_df

,age,weight,height,sex,fatpercentage,bmi
0,58,73.425545,162.931325,1,41.130917,27.659097
1,58,73.425545,162.931325,1,41.130917,27.659097
2,58,73.425545,162.931325,1,41.130917,27.659097
3,58,73.425545,162.931325,1,41.130917,27.659097
4,58,73.425545,162.931325,1,41.130917,27.659097
...,...,...,...,...,...,...
191995,58,92.125633,166.763229,0,36.892197,33.126831
191996,58,92.125633,166.763229,0,36.892197,33.126831
191997,58,92.125633,166.763229,0,36.892197,33.126831
191998,58,92.125633,166.763229,0,36.892197,33.126831


In [ ]:
#https://www.analyticsvidhya.com/blog/2021/06/complete-guide-on-how-to-use-autoencoders-in-python/

In [ ]:
#num = user_params_df.values
#num = tf.convert_to_tensor(num)

In [13]:
from sklearn.preprocessing import MinMaxScaler 
X = MinMaxScaler().fit_transform(user_params_df)

In [14]:
#https://www.geeksforgeeks.org/ml-classifying-data-using-an-auto-encoder/

In [15]:
input_layer = Input(shape =(X.shape[1], ))

In [16]:
#building the autoencoder network
encoded = Dense(100, activation ='linear',
                activity_regularizer = regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation ='linear',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(25, activation ='linear',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(12, activation ='linear',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(4, activation ='linear')(encoded)
  
# Building the Decoder network
decoded = Dense(12, activation ='linear')(encoded)
decoded = Dense(25, activation ='linear')(decoded)
decoded = Dense(50, activation ='linear')(decoded)
decoded = Dense(100, activation ='linear')(decoded)
  
# Building the Output Layer
output_layer = Dense(X.shape[1], activation ='relu')(decoded)
output_layer = Dense(X.shape[1], activation ='relu')(decoded)

In [17]:
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer ="adam", loss ="binary_crossentropy" , metrics=['accuracy'])

In [18]:
autoencoder.fit(X, X, 
                batch_size = 20, epochs = 25, 
                shuffle = True, validation_split = 0.20)

Epoch 1/25
7680/7680 [==============================] - 9s 1ms/step - loss: 1.3425 - accuracy: 0.8709 - val_loss: 0.5129 - val_accuracy: 0.9550
Epoch 2/25
7680/7680 [==============================] - 8s 1ms/step - loss: 0.6533 - accuracy: 0.9309 - val_loss: 0.5213 - val_accuracy: 0.9150
Epoch 3/25
7680/7680 [==============================] - 8s 1ms/step - loss: 0.5146 - accuracy: 0.9401 - val_loss: 0.5108 - val_accuracy: 0.9933
Epoch 4/25
7680/7680 [==============================] - 8s 1ms/step - loss: 0.5193 - accuracy: 0.9234 - val_loss: 0.5252 - val_accuracy: 0.8700
Epoch 5/25
7680/7680 [==============================] - 8s 1ms/step - loss: 0.5239 - accuracy: 0.9130 - val_loss: 0.5419 - val_accuracy: 0.8150
Epoch 6/25
7680/7680 [==============================] - 8s 1ms/step - loss: 0.5182 - accuracy: 0.9248 - val_loss: 0.5158 - val_accuracy: 0.9100
Epoch 7/25
7680/7680 [==============================] - 8s 1ms/step - loss: 1.4319 - accuracy: 0.6059 - val_loss: 1.7784 - val_accuracy:

In [21]:
# Extracting the embedding
encoder = keras.Model(input_layer, encoded)
encoder.save('encoder-V2.h5', overwrite = True)

encoded_r = encoder.predict(X)
print(encoded_r)

[[-0.15971126  0.16893026 -0.30369845 -0.15407065]
 [-0.15971126  0.16893026 -0.30369845 -0.15407065]
 [-0.15971126  0.16893026 -0.30369845 -0.15407065]
 ...
 [-0.30509818 -0.12241039 -0.28520215  0.10905373]
 [-0.30509806 -0.12241033 -0.28520215  0.10905375]
 [-0.30509806 -0.12241033 -0.28520215  0.10905375]]


In [22]:

# Transforming to a CSV file using pandas
df = pd.DataFrame(encoded_r, columns = ["Attribute 1", "Attribute 2", "Attribute 3", "Attribute 4"])
print(df)

        Attribute 1  Attribute 2  Attribute 3  Attribute 4
0         -0.159711      0.16893    -0.303698    -0.154071
1         -0.159711      0.16893    -0.303698    -0.154071
2         -0.159711      0.16893    -0.303698    -0.154071
3         -0.159711      0.16893    -0.303698    -0.154071
4         -0.159711      0.16893    -0.303698    -0.154071
...             ...          ...          ...          ...
191995    -0.305098     -0.12241    -0.285202     0.109054
191996    -0.305098     -0.12241    -0.285202     0.109054
191997    -0.305098     -0.12241    -0.285202     0.109054
191998    -0.305098     -0.12241    -0.285202     0.109054
191999    -0.305098     -0.12241    -0.285202     0.109054

[192000 rows x 4 columns]


In [ ]:
#import tensorflow as tf
#encoder = tf.keras.models.load_model("encoder.h5", compile = False)
#test_output = encoder.predict([[59, 76.350691, 165.034328, 1, 41.809198, 28.032665]])


In [23]:
df.to_csv("R_values_V2.csv", index=False)